In [3]:
import pandas as pd
import numpy as np

In [4]:
coded_df = pd.read_csv('coded_df.csv')
coded_df.head()

,RESP_ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,BD1-4,...,MI4-5oth,MI6.1-3,MI7-4,MI9-4,i1_1a,i1_1b,i1_1c,i4_1,i5_1,i6_1
0,12477972045,263393441,44264.931690,44264.979861,163.116.193.114,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN
1,12477891999,263393441,44264.930752,44264.937627,163.116.193.117,NaN,NaN,NaN,NaN,Yes,...,NaN,1.0,4.0,4.0,1.0,1.0,1.0,0.0,1.0,1.0
2,12477891089,263393441,44264.930382,44264.964699,163.116.193.117,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,NaN
3,12477471535,263393441,44264.793819,44264.882963,101.128.127.169,NaN,NaN,NaN,NaN,Yes,...,"Multiple: activity design, start up, and impl...",2.0,NaN,4.0,1.0,1.0,1.0,1.0,1.0,1.0
4,12477102564,263393441,44264.699907,44264.710116,72.234.176.36,NaN,NaN,NaN,NaN,Yes,...,NaN,3.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,0.0


In [12]:
# check if roles are consistent between BD4-2 and MI3-4
coded_df[['BD4-2','MI3-4']]

,BD4-2,MI3-4
0,3.0,NaN
1,1.0,1.0
2,3.0,NaN
3,1.0,1.0
4,2.0,5.0
...,...,...
89,1.0,5.0
90,NaN,NaN
91,4.0,4.0
92,1.0,1.0
